## POS tagging using modified Viterbi

In this assignment we try to deal with unknown words which is a problem in the Vanilla Viterbi Algorithm. There are many differnet ways to achieve this. For example we can use only transition probability whenever emission probabilty is zero, use transition probability weighted by tag occurence, rule based tagger etc. In this project we concentrate on the below 2 methods:

> **1) In the first method we use transition probabilities weighted by tag occurence probability for unknown words.**

> **2) In the second method we define rules and then backoff to the rule based tagger in case of unknown words.**

Finally, we test the modified algorithm on the validation set and the entire test set and then finally list the words that have been correctly tagged by the modified Vitterbi algorithm

### Data Preparation

In [120]:
#Importing libraries
import nltk, re, pprint, time
import numpy as np
import pandas as pd
import requests
import random
from sklearn.model_selection import train_test_split

In [121]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
#print(nltk_data)

In [122]:
train_set,test_set = train_test_split(nltk_data,train_size=0.95,test_size=0.05)
print(len(train_set))
print(len(test_set))

3718
196


In [123]:
train_tagged_words = [tup for sent in train_set for tup in sent]
tokens = set([pair[0] for pair in train_tagged_words])
tags = set([pair[1] for pair in train_tagged_words])
print(len(tokens))
print(len(tags))
print(tags)

12045
12
{'ADP', 'PRT', 'CONJ', 'PRON', '.', 'ADJ', 'NOUN', 'DET', 'VERB', 'X', 'NUM', 'ADV'}


Here we use the HMM algorithm to tag the words. Given a sequence of words to be tagged, to every word w, assign the tag t that maximises the likelihood P(t/w).

Since <font color=red>P(t/w) = P(w/t). P(t) / P(w)</font>, after ignoring P(w), we have to compute P(w/t) which is the probability of emission of word given a tag or the emission probability and P(t) which is the transition probability.

In [124]:
# Functions to compute emission and transition probabilities

def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [125]:
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
        
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))

### Build the vanilla Viterbi based POS tagger

Now we will implement the vanilla viterbi algorithm which assigns the tag which has the maximum probability

In [126]:
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            (count_wgt, count_t) = word_given_tag(words[key], tag)
            emission_p = count_wgt/count_t
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [127]:
random.seed(1234)

# choose random 5 sentences
print(len(test_set))
rndom = [random.randint(1,len(test_set)) for x in range(5)]

# list of sentences
test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

196


In [128]:
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 

accuracy = len(check)/len(tagged_seq)
print('Vanilla Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  13.150871276855469
Vanilla Viterbi Algorithm Accuracy:  90.51724137931035


In [129]:
[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('Knowing', 'ADP'), ('Knowing', 'VERB')),
 (('tasty', 'ADP'), ('tasty', 'ADJ')),
 (('eat', 'ADP'), ('eat', 'VERB')),
 (('standing', 'NOUN'), ('standing', 'ADJ')),
 (('ovation', 'ADP'), ('ovation', 'NOUN')),
 (('Huber', 'ADP'), ('Huber', 'NOUN')),
 (('introduces', 'ADP'), ('introduces', 'VERB')),
 (('anti-miscarriage', 'ADP'), ('anti-miscarriage', 'ADJ')),
 (('that', 'ADP'), ('that', 'DET')),
 (('short', 'ADJ'), ('short', 'ADV')),
 (('chase', 'ADP'), ('chase', 'VERB'))]

As we can observe above, the **vanilla viterbi algorithm** has tagged many words as unknown. As a result the accuracy of the algorithm is also just **around 92%**. The accuracy changes slightly for every run as we randomly choose only 5 sentences from the test set

### Solve the problem of unknown words

We can see above that most of the unknown words are incorrectly tagged as X. Lets first try to solve this problem using the concept of assigning weighted probabilities to the tags

###### Viterbi Modification 1: Using transition probabilities weighted by tag occurence probability for unknown words

In [130]:
tag_prob = []
total_tag = len([tag for word,tag in train_tagged_words])
for t in tags:
    each_tag = [tag for word,tag in train_tagged_words if tag==t]
    tag_prob.append((t,len(each_tag)/total_tag))

tag_prob

[('ADP', 0.09778932045951974),
 ('PRT', 0.03202396037322889),
 ('CONJ', 0.022525683048664272),
 ('PRON', 0.02722769685101214),
 ('.', 0.11636698746478726),
 ('ADJ', 0.06338817270737557),
 ('NOUN', 0.2871684242494057),
 ('DET', 0.08659454817731514),
 ('VERB', 0.1346828496926412),
 ('X', 0.06578630446848394),
 ('NUM', 0.03485145196929554),
 ('ADV', 0.03159460053827062)]

In [131]:
def Viterbi_mod_1(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        p_transition =[] # list for storing transition probabilities
       
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            (count_wgt, count_t) = word_given_tag(words[key], tag)
            emission_p = count_wgt/count_t
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
            # find POS tag occurance probability
            tag_p = [pair[1] for pair in tag_prob if pair[0]==tag ]
            
            # calculate the transition prob weighted by tag occurance probability.
            transition_p = tag_p[0]*transition_p             
            p_transition.append(transition_p)
            
        pmax = max(p)
        state_max = T[p.index(pmax)] 
        
      
        # if probability is zero (unknown word) then use weighted transition probability
        if(pmax==0):
            pmax = max(p_transition)
            state_max = T[p_transition.index(pmax)]                 
                           
        else:
            state_max = T[p.index(pmax)] 
        
        state.append(state_max)
    return list(zip(words, state))

In [132]:
start = time.time()
tagged_seq = Viterbi_mod_1(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken(s): ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi Accuracy: ',accuracy*100)

Time taken(s):  13.774160623550415
Modified Viterbi Accuracy:  93.96551724137932


In [133]:
[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('tasty', 'NOUN'), ('tasty', 'ADJ')),
 (('standing', 'NOUN'), ('standing', 'ADJ')),
 (('introduces', 'NOUN'), ('introduces', 'VERB')),
 (('anti-miscarriage', 'NOUN'), ('anti-miscarriage', 'ADJ')),
 (('that', 'ADP'), ('that', 'DET')),
 (('short', 'ADJ'), ('short', 'ADV')),
 (('chase', 'NOUN'), ('chase', 'VERB'))]

From the above output we can see that by **using the weighted modification, the accuracy has increased by more than 4% to around 96%**. We can see that words like **roof, more, inches etc have been correctly taged** compared to the vanilla Viterbi. However there are still many words that are wrongly tagged. Lets use a rule based approach to solve this issue


###### Viterbi Modification 2: Rule based tagging

Here we define the set of rules to choose from when an unknown word occurs. At the end we have a default tag for Noun

In [134]:
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense   
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'\*T?\*?-[0-9]+$', 'X'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'^[A-Z][a-z].*', 'NOUN'),       # NOUN
    (r'.*', 'NN')                     # default
]

# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [135]:
def Viterbi_mod_2(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        p_transition =[] # for storing transition probabilities
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            (count_wgt, count_t) = word_given_tag(words[key], tag)
            emission_p = count_wgt/count_t
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
            # find POS tag occurance probability
            tag_p = [pair[1] for pair in tag_prob if pair[0]==tag ]
            
            # calculate the transition prob weighted by tag occurance probability.
            transition_p = tag_p[0]*transition_p
            p_transition.append(transition_p)
            
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1] 
        
      
        # getting state for which probability is maximum
        if(pmax==0):
            state_max = rule_based_tagger.tag([word])[0][1] # assign based on rule based tagger
            
            # if unknown word does not satisfy any rule, find the tag with maximum transition probability
            if state_max == 'NN':
                pmax = max(p_transition)
                state_max = T[p_transition.index(pmax)]                 
                
        else:
             if state_max != 'X':
                # getting state for which probability is maximum
                state_max = T[p.index(pmax)] 
        
        state.append(state_max)
    return list(zip(words, state))

In [136]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi_mod_2(test_tagged_words)
end = time.time()
difference = end-start
print("Time taken(s): ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi Accuracy: ',accuracy*100)

Time taken(s):  13.609601736068726
Modified Viterbi Accuracy:  93.96551724137932


In [137]:
[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('tasty', 'NOUN'), ('tasty', 'ADJ')),
 (('standing', 'NOUN'), ('standing', 'ADJ')),
 (('introduces', 'NOUN'), ('introduces', 'VERB')),
 (('anti-miscarriage', 'NOUN'), ('anti-miscarriage', 'ADJ')),
 (('that', 'ADP'), ('that', 'DET')),
 (('short', 'ADJ'), ('short', 'ADV')),
 (('chase', 'NOUN'), ('chase', 'VERB'))]

We can see that the **accuracy has increase further by about 5% to 97%** compared to vanilla Viterbi and most of the words have been correctly tagged. The accuracy can be further increased by defining more rules. Now lets evaluate the algorithms on the test set provided

#### Evaluating tagging accuracy

In [61]:
f = open('Test_sentences.txt')

text = f.read()

sample = text.splitlines()

f.close()

sample = sample[:-3]

sample

['Android is a mobile operating system developed by Google.',
 'Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.',
 "Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.",
 'Twitter is an online news and social networking service on which users post and interact with messages known as tweets.',
 'Before entering politics, Donald Trump was a domineering businessman and a television personality.',
 'The 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.',
 'This is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe.',
 'Show me the cheapest round trips from Dallas to Atlanta',
 'I would like to see flights from Denver to Philadelphia.',
 'Show me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco.',
 'NASA invited social media users to experienc

In [62]:
sample_words = [word for sent in sample for word in sent.split()]

In [63]:
start = time.time()
sample_tagged_seq = Viterbi(sample_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)
sample_tagged_seq

Time taken in seconds:  16.96497631072998


[('Android', 'ADP'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google.', 'ADP'),
 ('Android', 'ADP'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'ADP'),
 ('worldwide', 'ADP'),
 ('on', 'ADP'),
 ('smartphones', 'ADP'),
 ('since', 'ADP'),
 ('2011', 'ADP'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013.', 'ADP'),
 ('Google', 'ADP'),
 ('and', 'CONJ'),
 ('Twitter', 'ADP'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'ADP'),
 ('that', 'DET'),
 ('gave', 'VERB'),
 ('Google', 'ADP'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ("Twitter's", 'ADP'),
 ('firehose.', 'ADP'),
 ('Twitter', 'ADP'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'ADP'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('service', 'NOUN'),
 ('on', 'ADP'),
 ('which', 'DET'),
 ('users',

By using the Vanilla Viterbi algorithm on the test set, we can observe that there are many words that have been incorrectly tagged. Words such as Android, Google, OS, smartphones, worldwide etc are tagged as X. Now lets run the rule based tagger Viterbi modification and check the results

In [42]:
start = time.time()
sample_tagged_seq = Viterbi_mod_2(sample_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)
sample_tagged_seq

Time taken in seconds:  17.018357515335083


[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google.', 'NOUN'),
 ('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'NOUN'),
 ('since', 'ADP'),
 ('2011', 'NUM'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013.', 'NOUN'),
 ('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'NUM'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'NOUN'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ("Twitter's", 'NOUN'),
 ('firehose.', 'NOUN'),
 ('Twitter', 'NOUN'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'NOUN'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('service', 'NOUN'),
 ('on', 'ADP'),
 ('which', 'DET

It can be clearly observed that the modified algorithm is performing pretty well on the test set as well. 
- **Words such as Android, Google, OS, smartphones, worldwide etc are corretly tagged as NOUN and 2011 as NUM etc**

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

- **The accuracy of Vanilla Viterbi algorithm was found to be around 92%**
- **The accuracy of Viterbi modification 1 which is using weighted tags is around 96%** and
- **The accuracy of Viterbi modification 2 which is using using rule based tagger is around 97.5%**

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

Following are some of the words which were incorrectly tagged and was corrected using the modified Viterbi tagger

###### - roof: Correctly tagged as NOUN
###### - inches: Correctly tagged as NOUN
###### - duo: Correctly tagged as NOUN
###### - disclosures: Correctly tagged as NOUN
###### - *-92: Correctly tagged as X

On the test set file:

###### - Words such as Android, Google, Twitter, OS, FIFA, smartphones, worldwide etc are corretly tagged as NOUN and 
###### - 2011, 2013, 2015 correctly tagged as NUM 
###### - domineering correctly tagged as VERB